# Yellow Taxi Data - Schema Analysis

In [1]:
# Define the base URL pattern
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{}.parquet"

# Define date range
start_date = "2009-01"
end_date = "2024-08"

In [2]:
! pip3 install duckdb

In [3]:
import duckdb

con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")

In [4]:
import pandas as pd
import duckdb
import time
import logging
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)

# Create a DuckDB connection
con = duckdb.connect()

# date range
dates = pd.date_range(start_date, end_date, freq="MS").strftime("%Y-%m").tolist()

# Generate the list of SQL commands
sql_commands = [
    f"SELECT * FROM parquet_schema('{base_url.format(date.replace('-', '-'))}')" for date in dates
]

def load_with_backoff(con, sql_command, max_retries=10, initial_delay=15, backoff_factor=2):
    """
    Load data using a SQL command with retry and backoff logic.

    Args:
        con (duckdb.DuckDBPyConnection): DuckDB connection object.
        sql_command (str): The SQL command to execute.
        max_retries (int): Maximum number of retries.
        initial_delay (int): Initial delay in seconds.
        backoff_factor (int): Factor by which the delay increases after each retry.

    Returns:
        pd.DataFrame: DataFrame containing the loaded data, or None if it failed.
    """
    delay = initial_delay
    for attempt in range(max_retries):
        try:
            # print(f"Executing SQL: {sql_command} (Attempt {attempt + 1})...")
            # print(".", end="", flush=True)
            return con.sql(sql_command).df()
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Failed to execute SQL, retrying in {delay} seconds... ({attempt + 1}/{max_retries})")
                time.sleep(delay)
                delay *= backoff_factor
            else:
                raise RuntimeException(f"Failed to execute SQL after {max_retries} retries: {e}")
    return None

# Load all SQL commands with progress bar
dfs = []
for sql in tqdm(sql_commands, desc="Loading Schemas"):
    df = load_with_backoff(con, sql)
    if df is not None:
        dfs.append(df)

Loading Schemas: 100%|██████████| 188/188 [00:16<00:00, 11.31it/s]


In [5]:
# Concatenate all DataFrames into one
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(final_df)} rows from {len(dfs)} files.")
else:
    print("No data loaded.")

Loaded 3738 rows from 188 files.


In [6]:
final_df

,file_name,name,type,type_length,repetition_type,num_children,converted_type,scale,precision,field_id,logical_type
0,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,schema,None,None,REQUIRED,18.0,None,NaN,NaN,NaN,None
1,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,vendor_name,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
2,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Pickup_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
3,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Dropoff_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
4,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Passenger_Count,INT64,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
3733,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,tolls_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3734,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,improvement_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3735,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,total_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3736,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,congestion_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None


In [7]:
def add_yyyy_mm_column(df):
  """Adds a new column 'yyyy_mm' to the DataFrame based on filename patterns"""

  df['yyyy_mm'] = df['file_name'].str.extract(r'(\d{4}-\d{2})')
  return df

# Apply the function to your DataFrame
final_df = add_yyyy_mm_column(final_df.copy())  # Operate on a copy to avoid modifying original data
final_df

,file_name,name,type,type_length,repetition_type,num_children,converted_type,scale,precision,field_id,logical_type,yyyy_mm
0,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,schema,None,None,REQUIRED,18.0,None,NaN,NaN,NaN,None,2009-01
1,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,vendor_name,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
2,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Pickup_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
3,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Dropoff_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
4,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Passenger_Count,INT64,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2009-01
...,...,...,...,...,...,...,...,...,...,...,...,...
3733,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,tolls_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3734,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,improvement_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3735,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,total_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3736,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,congestion_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08


Assume col names will be lowercased in VastDB

In [8]:
final_df['name'] = final_df['name'].str.lower()
final_df = final_df.loc[final_df['name'] != 'schema']

Create the summary report

In [9]:
schemas = (
    final_df.groupby("yyyy_mm", group_keys=False)
    .apply(lambda df: df[["name", "type"]].set_index("name").to_dict()["type"], include_groups=False)
)

schemas = schemas.to_dict()

schema_changes = []
previous_schema = None

# Get the initial and latest schema
initial_schema = schemas[min(schemas.keys())]
initial_schema = dict(sorted(initial_schema.items(), key=lambda x: x[0]))

latest_schema = schemas[max(schemas.keys())]
latest_schema = dict(sorted(latest_schema.items(), key=lambda x: x[0]))

In [10]:
for file_name, schema in schemas.items():
    if previous_schema is not None:
        added = set(schema.keys()) - set(previous_schema.keys())
        removed = set(previous_schema.keys()) - set(schema.keys())
        changed = {
            col: (previous_schema[col], schema[col])
            for col in set(previous_schema.keys()) & set(schema.keys())
            if previous_schema[col] != schema[col]
        }

        schema_changes.append({
            "yyyy_mm": file_name,
            "added_columns": list(added),
            "removed_columns": list(removed),
            "changed_columns": changed,
        })
    previous_schema = schema

changes_df = pd.DataFrame(schema_changes)
changes_df = changes_df.set_index('yyyy_mm')

pd.set_option("display.max_rows", None)

In [11]:
initial_schema

{'end_lat': 'DOUBLE',
 'end_lon': 'DOUBLE',
 'fare_amt': 'DOUBLE',
 'mta_tax': 'DOUBLE',
 'passenger_count': 'INT64',
 'payment_type': 'BYTE_ARRAY',
 'rate_code': 'DOUBLE',
 'start_lat': 'DOUBLE',
 'start_lon': 'DOUBLE',
 'store_and_forward': 'DOUBLE',
 'surcharge': 'DOUBLE',
 'tip_amt': 'DOUBLE',
 'tolls_amt': 'DOUBLE',
 'total_amt': 'DOUBLE',
 'trip_distance': 'DOUBLE',
 'trip_dropoff_datetime': 'BYTE_ARRAY',
 'trip_pickup_datetime': 'BYTE_ARRAY',
 'vendor_name': 'BYTE_ARRAY'}

In [12]:
print("\nSchema Drift Summary:")
changes_df


Schema Drift Summary:


,added_columns,removed_columns,changed_columns
yyyy_mm,,,
2009-02,[],[],{}
2009-03,[],[],{}
2009-04,[],[],{}
2009-05,[],[],{}
2009-06,[],[],{}
2009-07,[],[],{}
2009-08,[],[],{}
2009-09,[],[],{}
2009-10,[],[],{}


latest schema

In [13]:
latest_schema

{'airport_fee': 'DOUBLE',
 'congestion_surcharge': 'DOUBLE',
 'dolocationid': 'INT32',
 'extra': 'DOUBLE',
 'fare_amount': 'DOUBLE',
 'improvement_surcharge': 'DOUBLE',
 'mta_tax': 'DOUBLE',
 'passenger_count': 'INT64',
 'payment_type': 'INT64',
 'pulocationid': 'INT32',
 'ratecodeid': 'INT64',
 'store_and_fwd_flag': 'BYTE_ARRAY',
 'tip_amount': 'DOUBLE',
 'tolls_amount': 'DOUBLE',
 'total_amount': 'DOUBLE',
 'tpep_dropoff_datetime': 'INT64',
 'tpep_pickup_datetime': 'INT64',
 'trip_distance': 'DOUBLE',
 'vendorid': 'INT32'}

In [18]:
column_name_mapping = [
    { 'airport_fee':           { 'type': 'DOUBLE', 'source': [ 'airport_fee' ] }},
    { 'congestion_surcharge':  { 'type': 'DOUBLE', 'source': [ 'congestion_surcharge' ] }},
    { 'dolocationid':          { 'type': 'INT32',  'source': [ 'dolocationid' ] }},
    { 'dropoff_latitude':      { 'type': 'DOUBLE', 'source': [ 'dropoff_latitude', 'end_lat' ] }},
    { 'dropoff_longitude':     { 'type': 'DOUBLE', 'source': [ 'dropoff_longitude', 'end_lon' ] }},
    { 'extra':                 { 'type': 'DOUBLE', 'source': [ 'extra' ] }},
    { 'fare_amount':           { 'type': 'DOUBLE', 'source': [ 'fare_amt' ] }},
    { 'improvement_surcharge': { 'type': 'DOUBLE', 'source': [ 'improvement_surcharge' ] }},
    { 'mta_tax':               { 'type': 'DOUBLE', 'source': [ 'mta_tax' ] }},
    { 'passenger_count':       { 'type': 'INT64',  'source': [ 'passenger_count' ] }},
    { 'payment_type':          { 'type': 'INT64',  'source': [ 'payment_type' ] }},
    { 'pulocationid':          { 'type': 'INT32',  'source': [ 'pulocationid' ] }},
    { 'pickup_latitude':       { 'type': 'DOUBLE', 'source': [ 'pickup_latitude', 'start_lat' ] }},
    { 'pickup_longitude':      { 'type': 'DOUBLE', 'source': [ 'pickup_longitude', 'start_lon' ] }},
    { 'ratecodeid':            { 'type': 'INT64',  'source': [ 'rate_code' ] }},
    { 'store_and_fwd_flag':    { 'type': 'DOUBLE', 'source': [ 'store_and_forward' ] }},
    { 'surcharge':             { 'type': 'DOUBLE', 'source': [ 'surcharge' ] }},
    { 'tip_amount':            { 'type': 'DOUBLE', 'source': [ 'tip_amount', 'tip_amt' ] }},
    { 'tolls_amount':          { 'type': 'DOUBLE', 'source': [ 'tolls_amount', 'tolls_amt' ] }},
    { 'total_amount':          { 'type': 'DOUBLE', 'source': [ 'total_amount', 'total_amt' ] }},
    { 'tpep_dropoff_datetime': { 'type': 'INT64',  'source': [ 'trip_dropoff_datetime', 'dropoff_datetime' ] }},
    { 'tpep_pickup_datetime':  { 'type': 'INT64',  'source': [ 'trip_pickup_datetime', 'pickup_datetime' ] }},
    { 'trip_distance':         { 'type': 'DOUBLE', 'source': [ 'trip_distance' ] }},
    { 'vendorid':              { 'type': 'INT32',  'source': [ 'vendor_name', 'vendor_id' ] }},
]